<img src="https://i.imgur.com/ZeJluqa.png">

<center><h1>Great Barrier Reef - YOLO Full Understanding</h1></center>

# Introduction

> 🐠 **Note1**: Thank you to [Awsaf](https://www.kaggle.com/awsaf49) and his amazing notebook **[Great-Barrier-Reef: YOLOv5 train 🌊](https://www.kaggle.com/awsaf49/great-barrier-reef-yolov5-train)**, which introduced me and got me started onto this amazing Object Detection path. 🙏 I could not have done it without him, as this is my first time getting acquinted to YOLO (what is it, how to use it, how to code it). His 2 notebooks (the one already mentioned and **[Great-Barrier-Reef: YOLOv5 [infer] 🌊](https://www.kaggle.com/awsaf49/great-barrier-reef-yolov5-infer)**) are amazing starters onto this world.

> 🐠 **Note2**: Here is the [official YOLOv5 user notebook on Kaggle by Ultralytics](https://www.kaggle.com/ultralytics/yolov5). It contains a full tutorial on how to get started. 

## What is YOLO?

🐠 **YOLO**: You Only Look Once (contrary to my initial beliefs of thinking it was related to You-Only-Live-Once 😁)

🐠 **What is it?** YOLO it's a very simple and fast algorithm that recognizes objects within an image in real time. It is made up by a single CNN and requires only one forward pass through the neural network in order to identify the objects.

🐠 **How does it work?**

1. The image is split in a grid that has the same dimension for each "tile".
2. We add the bounding boxes that identify each object. The bbox has the following format: `[width, height, class, bx, by]`, where `[bx, by]` represents the center of the object.
3. Intersection Over Union: this technique is used so the bounding box "catches" the object fully (and doesn't leave any part of it uncovered, neither it is too large for the object). The `IOU=1` if the predicted and actual box are identical.

<center><img src="https://i.imgur.com/Ce1sfqj.png" width=700></center>

*Source: [here](https://www.section.io/engineering-education/introduction-to-yolo-algorithm-for-object-detection/)*

> **Disclaimer**: as this is my very first attempt at an Object detection task, I might be wrong in some cases. If you observe anything odd, please do address it in the comments ^^.

### ⬇ Libraries Below

In [ ]:
# Libraries
import os
import sys
import wandb
import torch
import time
import random
import shutil
import yaml
from tqdm import tqdm
import warnings
import cv2
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from IPython.display import display_html


# Environment check
warnings.filterwarnings("ignore")
os.environ["WANDB_SILENT"] = "true"
CONFIG = {'competition': 'greatReef', '_wandb_kernel': 'aot'}

# 🐝 Secrets
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb")

! wandb login $secret_value_0

# Custom colors
class color:
    S = '\033[1m' + '\033[94m'
    E = '\033[0m'
    
my_colors = ["#16558F", "#1583D2", "#61B0B7", "#ADDEFF", "#A99AEA", "#7158B7"]
print(color.S+"Current Directory"+color.E, os.getcwd())
print(color.S+"Notebook Color Scheme:"+color.E)
sns.palplot(sns.color_palette(my_colors))

### ⬇ Helper Functions Below

In [ ]:
# === 🐝 W&B ===
def save_dataset_artifact(run_name, artifact_name, path):
    '''Saves dataset to W&B Artifactory.
    run_name: name of the experiment
    artifact_name: under what name should the dataset be stored
    path: path to the dataset'''
    
    run = wandb.init(project='g2net', 
                     name=run_name, 
                     config=CONFIG, anonymous="allow")
    artifact = wandb.Artifact(name=artifact_name, 
                              type='dataset')
    artifact.add_file(path)

    wandb.log_artifact(artifact)
    wandb.finish()
    print("Artifact has been saved successfully.")
    
    
def create_wandb_plot(x_data=None, y_data=None, x_name=None, y_name=None, title=None, log=None, plot="line"):
    '''Create and save lineplot/barplot in W&B Environment.
    x_data & y_data: Pandas Series containing x & y data
    x_name & y_name: strings containing axis names
    title: title of the graph
    log: string containing name of log'''
    
    data = [[label, val] for (label, val) in zip(x_data, y_data)]
    table = wandb.Table(data=data, columns = [x_name, y_name])
    
    if plot == "line":
        wandb.log({log : wandb.plot.line(table, x_name, y_name, title=title)})
    elif plot == "bar":
        wandb.log({log : wandb.plot.bar(table, x_name, y_name, title=title)})
    elif plot == "scatter":
        wandb.log({log : wandb.plot.scatter(table, x_name, y_name, title=title)})
        
        
def create_wandb_hist(x_data=None, x_name=None, title=None, log=None):
    '''Create and save histogram in W&B Environment.
    x_data: Pandas Series containing x values
    x_name: strings containing axis name
    title: title of the graph
    log: string containing name of log'''
    
    data = [[x] for x in x_data]
    table = wandb.Table(data=data, columns=[x_name])
    wandb.log({log : wandb.plot.histogram(table, x_name, title=title)})

# Step 1. Data Prep

When using YOLOv5 we will have to work more with folders and basic *command lines*, vs the usual coding techniques that are encountered in other competitions.

By default, we are within the `/kaggle/working` directory. Whenever you want to `save` a file, the output goes here.

Other 2 directories within the `/kaggle/` directory are `lib` and `input`, where you can find the datasets we are using.

🐠 The idea here is to create 2 more files: 
* `/kaggle/images` - empty folder where we will store our training images
* `/kaggle/labels` - empty folder where we will store out labels (or annotations) found within these images

In [ ]:
print(color.S+"-Directory Structure-"+color.E)
print(color.S+"Before:"+color.E, os.listdir("../"))

# Create 2 new folders
!mkdir -p '../images'
!mkdir -p '../labels'

print(color.S+"After:"+color.E, os.listdir("../"))

Now we can import our training dataset.

🐠 The `train` file below is an exact copy of the one within the `tensorflow-great-barrier-reef` dataset and a few more columns that were created [in my first notebook](https://www.kaggle.com/andradaolteanu/greatbarrierreef-full-guide-to-bboxaugmentation):
* **`no_annotations`**: number of bboxes found within the image
* **`path`**: full path to the `tensorflow-great-barrier-reef` image
* **`f_annotations`**: formated annotations created for image augmentation
* **`path_images` & `path_labels`**: full paths to the directories we have created above
* **`width` & `height`**: the same all over, represent the metrics for the image
* **`coco_bbox`**: the simplified version of the `annotations` column. The *COCO format* is one of the many ways to annotate a bounding box and it has the format `[x, y, width, height]`.

In [ ]:
# Import the prepped train dataset
train = pd.read_csv("../input/2021-greatbarrierreef-prep-data/train.csv")
# Remove all images that have no bounding box (removing ~80% of data)
train = train[train["no_annotations"]>0].reset_index(drop=True)

train.sample(3, random_state=24)

# Step 2. Copy Images & Labels

Once we have our dataset and folders, we can proceed with the next step. Now we will copy the needed data (the images to train on and the labels) into these directories. We do this so we can have *writing access* on it.

## I. The Images

**Copy** from `../input/tensorflow-great-barrier-reef/train_images` to `../images`.

> **Note**: `shutil` library is used to copy files from one place to another.

In [ ]:
# Populate the ../images folder

for path in tqdm(train["path"].tolist()):
    split_path = path.split("/")

    # Retrieve the video id (0, 1, 2) and its frame number
    video_id = split_path[-2]
    video_frame = split_path[-1]

    # Create new image path
    path_image = f"../images/{video_id}_{video_frame}"
    
    # Copy file from source (competition data) to destination (our new folder)
    shutil.copy(src=path, dst=path_image)

In [ ]:
# Glimpse of images folder now:
print(color.S+"Sample of 3 images from the ../images/ folder:"+color.E, os.listdir("../images")[:3])

plt.figure(figsize=(10, 10))
img_sample = cv2.imread("../images/video_1_6258.jpg")
img_sample = cv2.cvtColor(img_sample, cv2.COLOR_BGR2RGB)
plt.imshow(img_sample)
plt.axis("off");

## II. The Labels

### COCO2YOLO

Before copying the labels we need to create a function that **converts** the COCO bboxes to YOLO bboxes.

Hence, we need to go from `[xmin, ymin, w, h]` to the corresponding yolo format `[xmid, ymid, w, h]`.

<center><img src="https://i0.wp.com/prabhjotkaurgosal.com/wp-content/uploads/2021/03/image.png?resize=1536%2C419&ssl=1" width=900></center>

*Source: [here](https://prabhjotkaurgosal.com/weekly-learnings/weekly-learning-blogs/)*

In [ ]:
def coco2yolo(image_height, image_width, bboxes):
    """
    Converts a coco annotation format [xmin, ymin, w, h] to 
    the corresponding yolo format [xmid, ymid, w, h]
    
    image_height: height of the original image
    image_width: width of the original image
    bboxes: coco boxes to be converted
    return :: 
    
    inspo: https://www.kaggle.com/awsaf49/great-barrier-reef-yolov5-train
    """
    
    bboxes = np.array(bboxes).astype(float)
    
    # Normalize xmin, w
    bboxes[:, [0, 2]]= bboxes[:, [0, 2]]/ image_width
    # Normalize ymin, h
    bboxes[:, [1, 3]]= bboxes[:, [1, 3]]/ image_height
    
    # Converstion (xmin, ymin) => (xmid, ymid)
    bboxes[:, [0, 1]] = bboxes[:, [0, 1]] + bboxes[:, [2, 3]]/2
    
    # Clip values (between 0 and 1)
    bboxes = np.clip(bboxes, a_min=0, a_max=1)
    
    return bboxes

In [ ]:
# --- Example ---
bbox_example = [[559, 213, 50, 32], [679, 223, 10, 100]]

print(color.S+"From COCO: "+color.E, bbox_example)
print(color.S+"to YOLO:"+color.E, 
      coco2yolo(image_height=720, 
                image_width=1280, 
                bboxes=bbox_example))

### Copy Labels

The **YOLO** template requires a little bit more than just the new YOLO bboxes. You can [read this article](https://towardsdatascience.com/image-data-labelling-and-annotation-everything-you-need-to-know-86ede6c684b1) to find out a little bit more about how the process works.

🐠 **The Summary**: In the YOLO labeling format, a `.txt` file with the same name is created for each image file in the same directory. Each `.txt` file contains the annotations for the corresponding image file, that is:
* *object class* - not applicable in our case, so it will be always set to 0
* *YOLO bbox* - the YOLO bboxes we have just created

In [ ]:
# Populate the ../labels folder
yolo_bboxes = []

for k in tqdm(range(len(train))):
    
    row_data = train.iloc[k, :]
    height = row_data["height"]
    width = row_data["width"]
    coco_bbox = eval(row_data["coco_bbox"])
    len_bbox = row_data["no_annotations"]
    
    # Create file and write in it
    with open(row_data["path_labels"], 'w') as file:
        
        # In case there is an image with no present annotation
        if len_bbox == 0: 
            file.write("")
            continue
            
        # Convert coco format to yolo format
        yolo_bbox = coco2yolo(height, width, coco_bbox)
        yolo_bboxes.append(yolo_bbox)
        
        # Write annotations in file
        for i in range(len_bbox):
            annot = ["0"] + \
                    yolo_bbox[i].astype(str).tolist() + \
                    ([""] if i+1 == len_bbox else ["\n"])
            
            annot = " ".join(annot).strip()
            file.write(annot)
            

# Add yolo boxes to dataframe
train["yolo_bbox"] = yolo_bboxes

In [ ]:
# Glimpse of labels folder now:
print(color.S+"Sample of 3 labels from the ../labels/ folder:"+color.E, os.listdir("../labels")[:3], "\n")

# Let's read the files
f1 = open('../labels/video_1_4238.txt', 'r')
f2 = open('../labels/video_1_5315.txt', 'r')
f3 = open('../labels/video_0_1006.txt', 'r')

# How the .txt files look?
print(color.S+"File1: "+color.E, f1.read())
print(color.S+"File2: "+color.E, f2.read())
print(color.S+"File3: "+color.E, f3.read())

## III. Did we do a good job?

We have created 2 folders: the images and the corresponding labels (annotations).

Let's test a few images to see if the look good.

In [ ]:
# Retrieve a sample of data
images = os.listdir("/kaggle/images")[6:12]
vid_id = [im.split("_")[1] for im in images]
seq_id = [im.split("_")[2].split(".")[0] for im in images]

# Plot
fig, axs = plt.subplots(2, 3, figsize=(23, 10))
axs = axs.flatten()
fig.suptitle(f"Sample of images and YOLO bounding boxes", fontsize = 20)

for k in range(6):
    
    # Get the data
    im = cv2.imread(f"/kaggle/images/{images[k]}")
    im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    dh, dw, _ = im.shape
    txt = open(f"/kaggle/labels/video_{vid_id[k]}_{seq_id[k]}.txt", "r").read().split(" ")[1:]
    no_boxes = int(len(txt)/4)
    
    # Draw boxes
    i = 0
    while i < no_boxes:
        i = i+4
        box = txt[:i][-4:]
        
        # Src: https://github.com/pjreddie/darknet/blob/810d7f797bdb2f021dbe65d2524c2ff6b8ab5c8b/src/image.c#L283-L291
        # from YOLO to COCO
        x, y, w, h = box
        x, y, w, h = float(x), float(y), float(w), float(h)

        l = int((x - w / 2) * dw)
        r = int((x + w / 2) * dw)
        t = int((y - h / 2) * dh)
        b = int((y + h / 2) * dh)

        if l < 0: l = 0
        if r > dw - 1: r = dw - 1
        if t < 0: t = 0
        if b > dh - 1: b = dh - 1

        cv2.rectangle(im, (l, t), (r, b), (255,0,0), 3)
        
    # Show image with bboxes
    axs[k].set_title(f"Sample {k}", fontsize = 14)
    axs[k].imshow(im)
    axs[k].set_axis_off()

plt.tight_layout()
plt.show()

# Step 3. YOLO Configuration

## I. Splitting the Data

As I have seen within discussions that it is best to group the data by taking into account that the images are actually *videos*. That being said, you don't want to train you model on image `i` and then test it on image `i+1`, as this could be translated to **data leakage**.

It's like showing a child a cat now and then moving it a little bit to the left and asking the child what animal it is. Of course they'll respond correctly! But if you show a child a cat and after an hour you show another cat in another environment ... then is when you'll actually test their capabilities of learning.

🐠 Hence, I have splitted the data simply into 2 parts:
* `train_data`: images from video_0 (2,143 observations) and video_2 (677 observations)
* `test_data`: images from video_1 (2,099 observations)

In [ ]:
# Use Video 0 and 2 for training and 1 for validation
train_data = train[train["video_id"].isin([0, 2])]
test_data = train[train["video_id"].isin([1])]

# Get path to images & labels
train_images = list(train_data["path_images"])
train_labels = list(train_data["path_labels"])

test_images = list(test_data["path_images"])
test_labels = list(test_data["path_labels"])

print(color.S+"Train Length:"+color.E, len(train_data), "\n" +
      color.S+"Test Length:"+color.E, len(test_data))

## II. Configuration Setup

This part will require us to:
* Create a "../working/train_images.txt"
* Create a "../working/test_images.txt" - these will be populated later
* Create a `yaml` configuration file

In [ ]:
print(color.S+"../working BEFORE:"+color.E, os.listdir("../working"))

# Create train and test path data
with open("../working/train_images.txt", "w") as file:
    for path in train_images:
        file.write(path + "\n")
        
with open("../working/test_images.txt", "w") as file:
    for path in test_images:
        file.write(path + "\n")


# Create configuration
config = {'path': '/kaggle/working',
          'train': '/kaggle/working/train_images.txt',
          'val': '/kaggle/working/test_images.txt',
          'nc': 1,
          'names': ['cots']}

with open("../working/cots.yaml", "w") as file:
    yaml.dump(config, file, default_flow_style=False)

        
print(color.S+"../working AFTER:"+color.E, os.listdir("../working"))

# Step 4. YOLOv5 Training

## I. YOLOv5

YOLO's first model was released in 2016, followed by YOLOv2 in 2017 and YOLOv3 in 2018. In 2020 [Joseph Redmon](https://machinelearningknowledge.ai/introduction-to-yolov5-object-detection-with-tutorial/#Introduction) stepped out from the project and his work was further improved by Alexey Bochkovskiy who produced YOLOv4 in 2020.

**YOLOv5** is the next controversial member of the YOLO family released in 2020 by the company Ultranytics just a few days after YOLOv4 ([source here](https://machinelearningknowledge.ai/introduction-to-yolov5-object-detection-with-tutorial/#Introduction)). It is controversial because there has never been any paper released to back up the model, nevertheless it works!

<center><img src="https://machinelearningknowledge.ai/wp-content/uploads/2021/06/YOLOv5-Architecture.jpg" width=700></center>

*[Source here](https://www.researchgate.net/publication/349299852_A_Forest_Fire_Detection_System_Based_on_Ensemble_Learning)*

## II. YOLOv5 Setup

To use the model we need to have the following:
* Yolov5 Repository ([available in this dataset by Awsaf](https://www.kaggle.com/awsaf49/yolov5-lib-ds))
* Python 3
* PyTorch
* CUDA

> **Note**: put `/kaggle/` and not `../` otherwise won't work.

In [ ]:
# ---> YOLOv5 install <---
%cd /kaggle/working     
!cp -r /kaggle/input/yolov5-lib-ds /kaggle/working/yolov5     
%cd yolov5     
%pip install -qr requirements.txt   

from yolov5 import utils
display = utils.notebook_init()

## III. Training

* 🐝 **Note**: YOLOv5 **connects automatically to your W&B account** and tracks the runs and progress there, so you do not need to log in anything during training.

🐠 Within the training cell:
* specify the dataset - `cots.yaml`
* batch size
* image size
* pretrained yolov5 weights (`yolov5s.pt`, `yolov5m.pt` etc.)

**There are multiple models you could try:**

<center><img src="https://github.com/ultralytics/yolov5/releases/download/v1.0/model_comparison.png" width=700></center>

*[Source here](https://docs.ultralytics.com/tutorials/train-custom-datasets/) - [full table with all available options here](https://github.com/ultralytics/yolov5#pretrained-checkpoints)*

In [ ]:
# --- PARAMETERS ---
# These are just small samples, so the notebook runs faster
SIZE = 500
BATCH_SIZE = 4
EPOCHS = 3
MODEL = "yolov5s"
WORKERS = 1
PROJECT = "GreatBarrierReef"
RUN_NAME = f"{MODEL}_size{SIZE}_epochs{EPOCHS}_batch{BATCH_SIZE}_simple"
# ------------------

In [ ]:
# Training - train.py can be found in yolov5 directory
!python train.py --img {SIZE}\
                --batch {BATCH_SIZE}\
                --epochs {EPOCHS}\
                --data /kaggle/working/cots.yaml\
                --weights {MODEL}.pt\
                --workers {WORKERS}\
                --project {PROJECT}\
                --name {RUN_NAME}\
                --exist-ok

## IV. Inspecting the results

All training results are saved to `../working/yolov5/runs/train/` with incrementing run directories (first run is `exp`, then `exp2`, `exp3` and so on).

> 🐝 **Note**: in this notebooks, `runs` is actually the folder `GreatBarrierReef`, as I am saving the logs during training within [my personal Dashboard for this competition here](https://wandb.ai/andrada/GreatBarrierReef?workspace=user-andrada). This way I can properly *name* the experiments - so, instead of having exp1, exp2 etc, I can have a proper name that will better indicate the experiment I am making. Also, **all data in this folder can be viewed in the W&B Dashboard**.

<center><img src="https://i.imgur.com/n8elExY.png" width=700></center>

🐠 Below it's a view of the new files created after training:

In [ ]:
# Run details
os.listdir(f"{PROJECT}/{RUN_NAME}")

Here is a sneak peak of how the folder looks locally on my workstation:

<center><img src="https://i.imgur.com/V5TzfyD.png" width=550></center>

🐠 I went ahead and saved the trained model here: `../input/2021-greatbarrierreef-prep-data`

You can also find it in [my dataset on this competition](https://www.kaggle.com/andradaolteanu/2021-greatbarrierreef-prep-data) in the folder [output].

In [ ]:
# Remove training data files
!rm -r '/kaggle/images'
!rm -r '/kaggle/labels'

# Step 5. Inference

## I. Competition metric

🐠 **F2** metric is computed based on:
* **precision**: which looks at how accurately we have identified the positives (the false positives should be minimum)
* **recall**: which looks at how accurately we have identified the true positives and true negaives in general (surprising as many as possible)

<center><img src="https://i.imgur.com/tfjRMDA.png" width=700></center>

This competiton uses [the F2 Score at different intersection over union (IoU) thresholds](https://www.kaggle.com/c/tensorflow-great-barrier-reef/overview/evaluation). This adjusted F2 puts in balance and is **in favor for recall rather than precision**, meaning that we want to have as many true cases of COTS catched, so we don't mind a few false positives here and there as well.

> 🐠 **Note**: Each predicted bounding box should have a *confidence level*, meaning that we need to provide a *certainty* that within the bounding box is actually a COTS or not.

## II. Loading the model

> 🐠 **Note**: I will use [this dataset](reef_baseline_fold12) by [sheep](https://www.kaggle.com/steamedsheep) to do the inference. 🙏

In [ ]:
# Change our position within the directory back
%cd /kaggle/working

In [ ]:
# --- Trained Model ---
MODEL_PATH = "../input/reef-baseline-fold12/l6_3600_uflip_vm5_f12_up/f1/best.pt"

# Load the model
model = torch.hub.load("../input/yolov5-lib-ds", "custom",
                       path=MODEL_PATH,
                       source='local', force_reload=True)

# BoundingBox Confidence
model.conf = 0.01
# Intersection Over Union
model.iou = 0.5

In [ ]:
# Create Ultralytics directory
!mkdir -p /root/.config/Ultralytics
# Copy folder to root
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

## III. Prediction

In [ ]:
import greatbarrierreef

# Initialize the environment
env = greatbarrierreef.make_env()
# Iterator that loops through the submission dataset
# !!! you can run this cell only once
iter_test = env.iter_test()

🐠 Looking at the requirements, the sample prediction must look as follows: `sample_prediction_df['annotations'] = '0.5 0 0 100 100'`. Hence, the submitted bounding box should have the format `'conf x y width height'`.

If there are multiple boxes in one image they would look like: `'conf1 x1 y1 width1 height1 conf2 x2 y2 width2 height2'`.

In [ ]:
# !!! you can run this cell only once

# Loop through the test file
for k, (image, sample_prediction_df) in enumerate(tqdm(iter_test)):
    
    annotation = ""
    prediction = model(image, size=3600, augment=True)
    print(color.S+"Prediction Object:"+color.E, prediction.pandas())
    print(color.S+"Bounding Boxes:"+color.E, prediction.pandas().xyxy[0])
    print(color.S+"Shape:"+color.E, prediction.pandas().xyxy[0].shape[0])
    
    if prediction.pandas().xyxy[0].shape[0] == 0:
        annotation = ""
    else:
        for k, row in prediction.pandas().xyxy[0].iterrows():
            if row.confidence > 0.15:
                conf = row.confidence
                x = int(row.xmin)
                y = int(row.ymin)
                width = int(row.xmax-row.xmin)
                height = int(row.ymax-row.ymin)
                annotation += "{} {} {} {} {}".format(conf, x, y, width, height)
    
    sample_prediction_df['annotations'] = annotation.strip(' ')
    
    # Register your predictions
    env.predict(sample_prediction_df)

<center><img src="https://i.imgur.com/0cx4xXI.png"></center>

### 🐝 W&B Dashboard

> My [W&B Dashboard](https://wandb.ai/andrada/GreatBarrierReef/workspace?workspace=user-andrada).

<center><video src="https://i.imgur.com/z3d7smf.mp4" width=800 controls></center>

<center><img src="https://i.imgur.com/knxTRkO.png"></center>

### My Specs

* 🖥 Z8 G4 Workstation
* 💾 2 CPUs & 96GB Memory
* 🎮 NVIDIA Quadro RTX 8000
* 💻 Zbook Studio G7 on the go